In [14]:
import pandas as pd
from datetime import datetime
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import joblib  # To save and load scaler

# Load dataset
df = pd.read_excel("WheatData2.xlsx")

# Parse date and extract time features
df['Date'] = pd.to_datetime(df['Date'])
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day

# Normalize macroeconomic features
features_to_normalize = ['CPI Food Items', 'PMS PRICE', 'Central Rate (US DOLL)', 'mpr']
scaler = MinMaxScaler()
df[features_to_normalize] = scaler.fit_transform(df[features_to_normalize])

# Save the fitted scaler for reuse
joblib.dump(scaler, 'scaler.pkl')

# Define commodity columns (C to K)
commodity_columns = df.columns[2:11].tolist()

# Dictionary to store trained models and their features
trained_models = {}

def train_all_commodities():
    for commodity in commodity_columns:
        df[commodity] = df[commodity].ffill().bfill()

        try:
            arima_model = ARIMA(df[commodity], order=(1, 1, 1))
            arima_result = arima_model.fit()
            df['Residuals'] = arima_result.resid
        except Exception as e:
            print(f" ARIMA failed for {commodity}: {e}")
            continue

        features = ['Residuals', 'year', 'month', 'day'] + features_to_normalize
        target = commodity

        model_data = df[features + [target]].dropna()
        X = model_data[features]
        y = model_data[target]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

        model = CatBoostRegressor(iterations=200, learning_rate=0.1, depth=6, random_state=42, verbose=0)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        #print(f" Evaluation for {commodity}")
        #print(f"   MAE: {mae:.2f}")
        #print(f"   MSE: {mse:.2f}")
        #print(f"   R² : {r2:.2f}")

        trained_models[commodity] = {
            'model': model,
            'features': features
        }

valid_locations = df['Location'].dropna().unique().tolist()

def predict_batch_prices(commodity_name, location, future_dates, macro_inputs_raw, export=False, file_name="batch_predictions.xlsx"):
    if commodity_name not in trained_models:
        print(f" '{commodity_name}' is not trained. Choose from:\n{list(trained_models.keys())}")
        return
    if location not in valid_locations:
        print(f" '{location}' is not a recognized location. Choose from:\n{valid_locations}")
        return
    # Load the same scaler used during training
    scaler = joblib.load('scaler.pkl')

    model_info = trained_models[commodity_name]
    model = model_info['model']
    features = model_info['features']

    predictions = []

    for date_str in future_dates:
        future_date = pd.to_datetime(date_str)

        # Construct input data with date and residual placeholder
        input_data = {
            'year': future_date.year,
            'month': future_date.month,
            'day': future_date.day,
            'Residuals': 0  # Residuals are 0 for future unknown values
        }
        # Validate raw macro inputs
        for feature in features_to_normalize:
            if feature not in macro_inputs_raw:
                print(f"Missing macroeconomic input: {feature}")
                return
        
        # Normalize using the saved scaler
        macro_df = pd.DataFrame([macro_inputs_raw])
        macro_scaled = scaler.transform(macro_df)
        macro_scaled_dict = dict(zip(features_to_normalize, macro_scaled[0]))


        # Merge scaled macro inputs into input data
        input_data.update(macro_scaled_dict)

        # Ensure feature order matches training
        input_df = pd.DataFrame([input_data])[features]

        predicted_price = model.predict(input_df)[0]

        predictions.append({
            'Commodity': commodity_name,
            'Location': location,
            'Date': future_date.date(),
            'Predicted Price': round(predicted_price, 2)
        })

    results_df = pd.DataFrame(predictions)
    print("\n📊 Batch Prediction Results")
    print(results_df)

    if export:
        results_df.to_excel(file_name, index=False)
        print(f"\n📁 Results exported to: {file_name}")

    return results_df


In [19]:
train_all_commodities()

predict_batch_prices(
    commodity_name="Alkama (Wheat)",
    location="Abuja",
    future_dates=["2025-12-01", "2026-01-01", "2026-02-01"],
    macro_inputs_raw={  # 🔁 CHANGED from macro_inputs
        'CPI Food Items': 946.5,
        'PMS PRICE': 978.36,
        'Central Rate (US DOLL)': 1118.35,
        'mpr': 23.4
    },
    export=False,
    file_name="Alkama_Kaduna_Forecast.xlsx"
)



📊 Batch Prediction Results
        Commodity Location        Date  Predicted Price
0  Alkama (Wheat)    Abuja  2025-12-01           874.55
1  Alkama (Wheat)    Abuja  2026-01-01           780.84
2  Alkama (Wheat)    Abuja  2026-02-01           789.08


,Commodity,Location,Date,Predicted Price
0,Alkama (Wheat),Abuja,2025-12-01,874.55
1,Alkama (Wheat),Abuja,2026-01-01,780.84
2,Alkama (Wheat),Abuja,2026-02-01,789.08
